In [1]:
import pennylane as qml
from scipy.sparse.linalg import eigsh
import numpy as np
from susy_qm import calculate_Hamiltonian

In [3]:
potential = 'DW'

In [9]:
cutoffs = [4,8,16]
for cutoff in cutoffs:
    print(cutoff, potential)
    H = calculate_Hamiltonian(cutoff, potential)
        
    eigenvalues, eigenvectors = np.linalg.eig(H)
    min_eigenvalue = min(eigenvalues.real)
    min_index = np.argmin(eigenvalues)
    min_eigenvalue = eigenvalues[min_index]
    min_eigenvector = np.asarray(eigenvectors[:, min_index])

    num_levels = 1
    min_ev = eigenvalues.argsort()[:num_levels]
    for e_lev in range(num_levels):
        eig_vec = np.asarray(eigenvectors[:, min_ev[e_lev]].real)
        eig_vec[np.abs(eig_vec) < 1e-9] = 0
        eig_vec = eig_vec.reshape((2, cutoff))
        print(eig_vec)
        print("#####################################")

4 DW
[[ 0.          0.          0.          0.        ]
 [ 0.96698015 -0.16164595 -0.18805279  0.05878883]]
#####################################
8 DW
[[ 0.78809287 -0.58201553  0.1393305   0.08947937 -0.09466353  0.02761835
   0.03154059 -0.04504704]
 [ 0.          0.          0.          0.          0.          0.
   0.          0.        ]]
#####################################
16 DW
[[ 7.90444362e-01 -5.83054050e-01  1.40808200e-01  8.43368377e-02
  -8.41227336e-02  2.15341878e-02  1.58155893e-02 -1.95095017e-02
   5.61182898e-03  6.36916635e-03 -6.49531964e-03  1.36568657e-04
   3.38516909e-03 -1.86669411e-03 -1.19433082e-03  2.07081883e-03]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]]
#####################################
